# Models and Database ORM

## Learning Objectives

By the end of this notebook, you will be able to:

1. Define Django models with various field types
2. Understand and create relationships between models
3. Create and apply database migrations
4. Perform CRUD operations using Django's ORM
5. Use QuerySets for filtering and retrieving data
6. Navigate related objects and perform lookups
7. Connect Django ORM concepts to raw SQL knowledge

---

## 1. Introduction to Django Models

A Django **model** is a Python class that represents a database table. Each attribute of the model represents a database column.

### Connecting to SQL Concepts

If you've worked with SQL (from the previous module), here's how Django ORM maps to SQL:

| SQL Concept | Django ORM Equivalent |
|-------------|----------------------|
| Table | Model class |
| Column | Model field |
| Row | Model instance |
| PRIMARY KEY | `id` field (auto-created) |
| FOREIGN KEY | `ForeignKey` field |
| SELECT | `objects.all()`, `objects.filter()` |
| INSERT | `Model.objects.create()` or `instance.save()` |
| UPDATE | Modify attributes and `instance.save()` |
| DELETE | `instance.delete()` |

## 2. Defining Models

Models are defined in your app's `models.py` file:

```python
# catalog/models.py
from django.db import models

class Author(models.Model):
    """Represents a book author."""
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    birth_date = models.DateField(null=True, blank=True)
    biography = models.TextField(blank=True)
    
    def __str__(self):
        return f"{self.first_name} {self.last_name}"
    
    class Meta:
        ordering = ['last_name', 'first_name']


class Book(models.Model):
    """Represents a book in the catalog."""
    title = models.CharField(max_length=200)
    author = models.ForeignKey(Author, on_delete=models.CASCADE)
    isbn = models.CharField('ISBN', max_length=13, unique=True)
    price = models.DecimalField(max_digits=6, decimal_places=2)
    published_date = models.DateField()
    in_stock = models.BooleanField(default=True)
    
    def __str__(self):
        return self.title
```

This would create the following SQL tables (simplified):

```sql
CREATE TABLE catalog_author (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    birth_date DATE NULL,
    biography TEXT
);

CREATE TABLE catalog_book (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title VARCHAR(200) NOT NULL,
    author_id INTEGER NOT NULL REFERENCES catalog_author(id),
    isbn VARCHAR(13) NOT NULL UNIQUE,
    price DECIMAL(6, 2) NOT NULL,
    published_date DATE NOT NULL,
    in_stock BOOLEAN DEFAULT TRUE
);
```

## 3. Field Types

Django provides many field types for different data:

In [ ]:
# Common Django field types (reference)

field_types = {
    # Text fields
    'CharField': 'Short text (requires max_length)',
    'TextField': 'Long text (no max_length needed)',
    'EmailField': 'Email with validation',
    'URLField': 'URL with validation',
    'SlugField': 'URL-friendly short label',
    
    # Numeric fields
    'IntegerField': 'Integer (-2147483648 to 2147483647)',
    'PositiveIntegerField': 'Integer (0 to 2147483647)',
    'FloatField': 'Floating-point number',
    'DecimalField': 'Fixed-precision decimal (max_digits, decimal_places)',
    
    # Boolean
    'BooleanField': 'True/False value',
    'NullBooleanField': 'True/False/None (deprecated, use BooleanField(null=True))',
    
    # Date and time
    'DateField': 'Date (YYYY-MM-DD)',
    'TimeField': 'Time (HH:MM:SS)',
    'DateTimeField': 'Date and time combined',
    'DurationField': 'Time duration (timedelta)',
    
    # File fields
    'FileField': 'File upload',
    'ImageField': 'Image upload (requires Pillow)',
    
    # Relationship fields
    'ForeignKey': 'Many-to-one relationship',
    'OneToOneField': 'One-to-one relationship',
    'ManyToManyField': 'Many-to-many relationship',
}

print("Django Field Types:")
print("=" * 60)
for field, description in field_types.items():
    print(f"{field:25} - {description}")

### Common Field Options

```python
# Field options
name = models.CharField(
    max_length=100,      # Required for CharField
    null=True,           # Database can store NULL (default: False)
    blank=True,          # Form validation allows empty (default: False)
    default='Unknown',   # Default value
    unique=True,         # Must be unique across all rows
    db_index=True,       # Create database index
    primary_key=True,    # Use as primary key
    verbose_name='Name', # Human-readable name
    help_text='Enter...',# Help text for forms
    choices=[            # Limit to specific choices
        ('draft', 'Draft'),
        ('published', 'Published'),
    ],
)
```

## 4. Model Relationships

Django supports three types of relationships:

### ForeignKey (Many-to-One)

```python
class Book(models.Model):
    # Many books can have one author
    author = models.ForeignKey(
        Author, 
        on_delete=models.CASCADE,  # Delete books if author is deleted
        related_name='books'        # Access via author.books.all()
    )
```

### OneToOneField

```python
class Profile(models.Model):
    # Each user has exactly one profile
    user = models.OneToOneField(
        User, 
        on_delete=models.CASCADE
    )
    bio = models.TextField()
```

### ManyToManyField

```python
class Book(models.Model):
    # A book can have many genres, a genre can have many books
    genres = models.ManyToManyField(
        Genre,
        related_name='books'
    )
```

### on_delete Options

| Option | Behavior |
|--------|----------|
| `CASCADE` | Delete related objects |
| `PROTECT` | Prevent deletion |
| `SET_NULL` | Set to NULL (requires null=True) |
| `SET_DEFAULT` | Set to default value |
| `DO_NOTHING` | Do nothing (may break integrity) |

## 5. Migrations

Migrations are Django's way of propagating model changes to the database schema.

### Creating and Applying Migrations

```bash
# After modifying models.py, create migration files
python manage.py makemigrations

# See what SQL will be executed
python manage.py sqlmigrate catalog 0001

# Apply migrations to database
python manage.py migrate

# Check migration status
python manage.py showmigrations
```

### Migration Best Practices

1. **Run `makemigrations` after every model change**
2. **Review migration files before applying** - they're in `app/migrations/`
3. **Commit migrations to version control** - they're part of your codebase
4. **Never edit migrations manually** unless you know what you're doing

## 6. Django ORM: CRUD Operations

The following examples should be run in the Django shell (`python manage.py shell`).

In [ ]:
# The code below demonstrates Django ORM operations
# In practice, run these in: python manage.py shell

# These are example commands - they won't run in a regular Python environment

orm_examples = """
# First, import your models
from catalog.models import Author, Book

# ===== CREATE =====

# Method 1: Create and save separately
author = Author(first_name='Jane', last_name='Austen')
author.save()

# Method 2: Create and save in one step
author = Author.objects.create(
    first_name='Charles',
    last_name='Dickens'
)

# SQL equivalent: INSERT INTO catalog_author (first_name, last_name) VALUES ('Charles', 'Dickens');


# ===== READ =====

# Get all objects
all_authors = Author.objects.all()
# SQL: SELECT * FROM catalog_author;

# Get a single object by primary key
author = Author.objects.get(pk=1)
# SQL: SELECT * FROM catalog_author WHERE id = 1;

# Get a single object by any field
author = Author.objects.get(last_name='Austen')
# Raises DoesNotExist if not found, MultipleObjectsReturned if > 1


# ===== UPDATE =====

# Update single object
author = Author.objects.get(pk=1)
author.first_name = 'Jane Austen'
author.save()
# SQL: UPDATE catalog_author SET first_name = 'Jane Austen' WHERE id = 1;

# Update multiple objects
Author.objects.filter(last_name='Unknown').update(biography='No biography available')
# SQL: UPDATE catalog_author SET biography = 'No biography available' WHERE last_name = 'Unknown';


# ===== DELETE =====

# Delete single object
author = Author.objects.get(pk=1)
author.delete()
# SQL: DELETE FROM catalog_author WHERE id = 1;

# Delete multiple objects
Author.objects.filter(last_name='Unknown').delete()
# SQL: DELETE FROM catalog_author WHERE last_name = 'Unknown';
"""

print(orm_examples)

## 7. QuerySets and Filtering

A **QuerySet** represents a collection of objects from the database. QuerySets are lazy - the database isn't accessed until you evaluate the QuerySet.

In [ ]:
# QuerySet filtering examples

queryset_examples = """
# ===== FILTERING =====

# filter() - returns QuerySet of matching objects
books = Book.objects.filter(in_stock=True)
# SQL: SELECT * FROM catalog_book WHERE in_stock = TRUE;

# exclude() - returns QuerySet of non-matching objects
books = Book.objects.exclude(in_stock=False)
# SQL: SELECT * FROM catalog_book WHERE NOT in_stock = FALSE;

# Chaining filters
books = Book.objects.filter(in_stock=True).filter(price__lt=20)
# SQL: SELECT * FROM catalog_book WHERE in_stock = TRUE AND price < 20;


# ===== FIELD LOOKUPS =====

# Exact match (default)
Book.objects.filter(title='Pride and Prejudice')
Book.objects.filter(title__exact='Pride and Prejudice')  # Same as above

# Case-insensitive match
Book.objects.filter(title__iexact='pride and prejudice')

# Contains (LIKE %value%)
Book.objects.filter(title__contains='Pride')
Book.objects.filter(title__icontains='pride')  # Case-insensitive

# Starts/ends with
Book.objects.filter(title__startswith='The')
Book.objects.filter(title__endswith='tion')

# Comparison operators
Book.objects.filter(price__gt=10)      # Greater than
Book.objects.filter(price__gte=10)     # Greater than or equal
Book.objects.filter(price__lt=20)      # Less than
Book.objects.filter(price__lte=20)     # Less than or equal

# Range (BETWEEN)
Book.objects.filter(price__range=(10, 20))
# SQL: WHERE price BETWEEN 10 AND 20

# In list
Book.objects.filter(id__in=[1, 2, 3])
# SQL: WHERE id IN (1, 2, 3)

# NULL check
Author.objects.filter(birth_date__isnull=True)
# SQL: WHERE birth_date IS NULL

# Date lookups
Book.objects.filter(published_date__year=2023)
Book.objects.filter(published_date__month=6)
Book.objects.filter(published_date__day=15)
"""

print(queryset_examples)

## 8. Ordering and Limiting Results

In [ ]:
# Ordering and limiting examples

ordering_examples = """
# ===== ORDERING =====

# Ascending order
Book.objects.order_by('title')
# SQL: SELECT * FROM catalog_book ORDER BY title ASC;

# Descending order (prefix with -)
Book.objects.order_by('-price')
# SQL: SELECT * FROM catalog_book ORDER BY price DESC;

# Multiple fields
Book.objects.order_by('author', '-published_date')
# SQL: ORDER BY author ASC, published_date DESC;

# Random order
Book.objects.order_by('?')


# ===== LIMITING (SLICING) =====

# First 5 results (LIMIT 5)
Book.objects.all()[:5]
# SQL: SELECT * FROM catalog_book LIMIT 5;

# Skip 5, get next 5 (OFFSET 5 LIMIT 5)
Book.objects.all()[5:10]
# SQL: SELECT * FROM catalog_book LIMIT 5 OFFSET 5;

# Get first object
Book.objects.first()
Book.objects.order_by('title').first()

# Get last object
Book.objects.last()

# NOTE: Negative indexing is NOT supported
# Book.objects.all()[-1]  # This will raise an error!


# ===== DISTINCT =====

# Remove duplicates
Author.objects.values('last_name').distinct()
# SQL: SELECT DISTINCT last_name FROM catalog_author;
"""

print(ordering_examples)

## 9. Related Objects and Lookups

Django ORM makes it easy to traverse relationships:

In [ ]:
# Related object examples

related_examples = """
# ===== FORWARD RELATIONSHIPS (ForeignKey) =====

# Get the related object
book = Book.objects.get(pk=1)
author = book.author  # Returns the Author object
print(author.first_name)


# ===== REVERSE RELATIONSHIPS =====

# Access books from author (using related_name or default _set)
author = Author.objects.get(pk=1)

# If related_name='books' was set:
all_books = author.books.all()

# If no related_name was set:
all_books = author.book_set.all()


# ===== FILTERING ACROSS RELATIONSHIPS =====

# Filter books by author's last name (double underscore)
Book.objects.filter(author__last_name='Austen')
# SQL: SELECT * FROM catalog_book 
#      JOIN catalog_author ON book.author_id = author.id
#      WHERE author.last_name = 'Austen';

# Filter authors who have books with price > 20
Author.objects.filter(books__price__gt=20)

# Chain relationships
Order.objects.filter(book__author__last_name='Dickens')


# ===== MANY-TO-MANY =====

# Add relationships
book = Book.objects.get(pk=1)
genre = Genre.objects.get(name='Fiction')
book.genres.add(genre)

# Remove relationships
book.genres.remove(genre)

# Clear all relationships
book.genres.clear()

# Set specific relationships (replaces existing)
book.genres.set([genre1, genre2])

# Get all related objects
book.genres.all()
genre.books.all()  # Reverse (if related_name='books')
"""

print(related_examples)

## 10. Aggregation and Annotation

In [ ]:
# Aggregation examples

aggregation_examples = """
from django.db.models import Count, Sum, Avg, Max, Min

# ===== AGGREGATE (returns dictionary) =====

# Average price of all books
Book.objects.aggregate(Avg('price'))
# Returns: {'price__avg': 15.50}
# SQL: SELECT AVG(price) FROM catalog_book;

# Multiple aggregations
Book.objects.aggregate(
    avg_price=Avg('price'),
    max_price=Max('price'),
    min_price=Min('price'),
    total_books=Count('id')
)


# ===== ANNOTATE (adds field to each object) =====

# Count books per author
authors = Author.objects.annotate(book_count=Count('books'))
for author in authors:
    print(f"{author.last_name}: {author.book_count} books")
# SQL: SELECT author.*, COUNT(book.id) as book_count
#      FROM catalog_author
#      LEFT JOIN catalog_book ON author.id = book.author_id
#      GROUP BY author.id;

# Filter on annotated values
Author.objects.annotate(book_count=Count('books')).filter(book_count__gt=5)

# Sum of book prices per author
Author.objects.annotate(total_value=Sum('books__price'))
"""

print(aggregation_examples)

## 11. Using the Django Shell

The Django shell is an interactive Python shell with Django configured:

```bash
# Start the shell
python manage.py shell

# Or with IPython (if installed)
python manage.py shell -i ipython
```

### Useful Shell Techniques

```python
# Import models
from catalog.models import Author, Book

# See the SQL that will be generated
print(Book.objects.filter(price__gt=10).query)

# Check if QuerySet will hit database
qs = Book.objects.filter(in_stock=True)
print(qs._result_cache)  # None if not evaluated yet

# Force evaluation
list(qs)  # Now it hits the database

# Count without loading all objects
Book.objects.filter(in_stock=True).count()

# Check if objects exist
Book.objects.filter(in_stock=True).exists()
```

## 12. ORM vs Raw SQL Comparison

Here's how common SQL operations translate to Django ORM:

In [ ]:
# SQL to Django ORM comparison

comparisons = [
    {
        'operation': 'Select all',
        'sql': 'SELECT * FROM books',
        'orm': 'Book.objects.all()'
    },
    {
        'operation': 'Select with WHERE',
        'sql': "SELECT * FROM books WHERE price > 10",
        'orm': 'Book.objects.filter(price__gt=10)'
    },
    {
        'operation': 'Select specific columns',
        'sql': 'SELECT title, price FROM books',
        'orm': "Book.objects.values('title', 'price')"
    },
    {
        'operation': 'Join tables',
        'sql': 'SELECT * FROM books JOIN authors ON books.author_id = authors.id',
        'orm': 'Book.objects.select_related("author")'
    },
    {
        'operation': 'Count',
        'sql': 'SELECT COUNT(*) FROM books',
        'orm': 'Book.objects.count()'
    },
    {
        'operation': 'Group By',
        'sql': 'SELECT author_id, COUNT(*) FROM books GROUP BY author_id',
        'orm': "Book.objects.values('author_id').annotate(count=Count('id'))"
    },
    {
        'operation': 'Order By',
        'sql': 'SELECT * FROM books ORDER BY title DESC',
        'orm': "Book.objects.order_by('-title')"
    },
    {
        'operation': 'LIKE',
        'sql': "SELECT * FROM books WHERE title LIKE '%Python%'",
        'orm': "Book.objects.filter(title__contains='Python')"
    },
]

print("SQL to Django ORM Comparison")
print("=" * 80)

for comp in comparisons:
    print(f"\n{comp['operation']}:")
    print(f"  SQL: {comp['sql']}")
    print(f"  ORM: {comp['orm']}")

---

## Exercises

### Exercise 1: Define a Model

Create a `Product` model for an e-commerce site with the following fields:
- `name` (string, max 200 characters, required)
- `description` (text, optional)
- `price` (decimal, 10 digits max, 2 decimal places)
- `quantity_in_stock` (positive integer, default 0)
- `created_at` (auto-set when created)
- `updated_at` (auto-set when updated)

<details>
<summary>Click to see solution</summary>

```python
# store/models.py
from django.db import models

class Product(models.Model):
    """Represents a product in the store."""
    name = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    quantity_in_stock = models.PositiveIntegerField(default=0)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return self.name
    
    class Meta:
        ordering = ['name']
```

</details>

### Exercise 2: Create Related Models

Create `Category` and `Review` models that relate to the `Product` model:
- A product belongs to one category (many-to-one)
- A product can have many reviews (one-to-many)
- Review should have: rating (1-5 integer), comment (text), reviewer_name, created_at

<details>
<summary>Click to see solution</summary>

```python
# store/models.py
from django.db import models
from django.core.validators import MinValueValidator, MaxValueValidator

class Category(models.Model):
    """Represents a product category."""
    name = models.CharField(max_length=100)
    description = models.TextField(blank=True)
    
    def __str__(self):
        return self.name
    
    class Meta:
        verbose_name_plural = 'categories'


class Product(models.Model):
    """Represents a product in the store."""
    name = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    quantity_in_stock = models.PositiveIntegerField(default=0)
    category = models.ForeignKey(
        Category, 
        on_delete=models.SET_NULL, 
        null=True,
        related_name='products'
    )
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return self.name


class Review(models.Model):
    """Represents a product review."""
    product = models.ForeignKey(
        Product, 
        on_delete=models.CASCADE,
        related_name='reviews'
    )
    rating = models.IntegerField(
        validators=[MinValueValidator(1), MaxValueValidator(5)]
    )
    comment = models.TextField()
    reviewer_name = models.CharField(max_length=100)
    created_at = models.DateTimeField(auto_now_add=True)
    
    def __str__(self):
        return f"{self.reviewer_name}'s review of {self.product.name}"
```

</details>

### Exercise 3: QuerySet Operations

Write Django ORM queries for the following:
1. Get all products with price less than $50
2. Get all products in the "Electronics" category
3. Get the 5 most expensive products
4. Get products that have at least one 5-star review
5. Get the average price of all products

<details>
<summary>Click to see solution</summary>

```python
from django.db.models import Avg, Count

# 1. Products under $50
cheap_products = Product.objects.filter(price__lt=50)

# 2. Products in Electronics category
electronics = Product.objects.filter(category__name='Electronics')

# 3. Top 5 most expensive
expensive = Product.objects.order_by('-price')[:5]

# 4. Products with 5-star reviews
five_star_products = Product.objects.filter(reviews__rating=5).distinct()

# 5. Average price
avg_price = Product.objects.aggregate(avg_price=Avg('price'))
print(avg_price)  # {'avg_price': Decimal('...')}
```

</details>

### Exercise 4: CRUD Operations

Write Django shell commands to:
1. Create a new category called "Books"
2. Create a product in that category
3. Update the product's price to $29.99
4. Add a review to the product
5. Delete all products with 0 stock

<details>
<summary>Click to see solution</summary>

```python
# In Django shell: python manage.py shell

from store.models import Category, Product, Review

# 1. Create category
books_cat = Category.objects.create(
    name='Books',
    description='Physical and digital books'
)

# 2. Create product
product = Product.objects.create(
    name='Python Crash Course',
    description='A hands-on, project-based introduction to Python',
    price=39.99,
    quantity_in_stock=50,
    category=books_cat
)

# 3. Update price
product.price = 29.99
product.save()
# Or: Product.objects.filter(pk=product.pk).update(price=29.99)

# 4. Add review
Review.objects.create(
    product=product,
    rating=5,
    comment='Excellent book for beginners!',
    reviewer_name='John Doe'
)

# 5. Delete products with 0 stock
deleted_count, _ = Product.objects.filter(quantity_in_stock=0).delete()
print(f"Deleted {deleted_count} products")
```

</details>

### Exercise 5: Complex Query with Annotation

Write a query to get all categories with:
- The number of products in each category
- The average price of products in each category
- Only include categories with at least 3 products
- Order by product count descending

<details>
<summary>Click to see solution</summary>

```python
from django.db.models import Count, Avg

categories = Category.objects.annotate(
    product_count=Count('products'),
    avg_price=Avg('products__price')
).filter(
    product_count__gte=3
).order_by('-product_count')

# Print results
for cat in categories:
    print(f"{cat.name}: {cat.product_count} products, avg price ${cat.avg_price:.2f}")

# Equivalent SQL:
# SELECT category.*, 
#        COUNT(product.id) as product_count,
#        AVG(product.price) as avg_price
# FROM store_category category
# LEFT JOIN store_product product ON category.id = product.category_id
# GROUP BY category.id
# HAVING COUNT(product.id) >= 3
# ORDER BY product_count DESC;
```

</details>

---

## Summary

In this notebook, you learned:

- **Models** are Python classes that define database tables
- Django provides many **field types** for different data
- **Relationships**: ForeignKey (many-to-one), OneToOneField, ManyToManyField
- **Migrations** sync model changes with the database
- **CRUD operations**: create(), save(), update(), delete()
- **QuerySets** are lazy and chainable
- **Field lookups** use double underscores (e.g., `price__gt=10`)
- **Aggregation** functions: Count, Sum, Avg, Max, Min
- **Annotation** adds computed fields to query results
- The Django ORM abstracts SQL while maintaining performance

## Next Steps

In the next notebook, we'll explore **Views and URL Routing**, where you'll learn how to:
- Create function-based and class-based views
- Configure URL patterns
- Handle HTTP requests and responses
- Use generic views for common patterns